<a href="https://colab.research.google.com/github/GwiHwan-Go/Detect_SC/blob/main/Depression_Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 수집한 데이터로 Word Graph 만들기

## 제공하는 데이터 설명

1. 데이터 설명 : DCinside 라는 커뮤니티에서 우울증 갤러리라는 곳의 게시물들의 '제목'들.
2. 데이터 오버뷰

  - 데이터 크기 : 2,470,236 개

  - 데이터 피쳐 : title (텍스트 데이터), author (작성자 id), date (작성 날짜)
  
3. 텍스트 데이터 : 미리 학습시킨 토크나이저로 토큰화한 데이터로 ' '로 구분되어 있음.




In [5]:
# 데이터 다운로드, 파일명은 sns_data.csv 
file_name = 'sns_data.csv'
!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-4qx3IRyXUtb_QSItiWyIHZIo9hcBttg' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-4qx3IRyXUtb_QSItiWyIHZIo9hcBttg" -O $file_name && rm -rf ~/cookies.txt


--2022-10-31 06:29:30--  https://docs.google.com/uc?export=download&confirm=t&id=1-4qx3IRyXUtb_QSItiWyIHZIo9hcBttg
Resolving docs.google.com (docs.google.com)... 142.250.159.113, 142.250.159.101, 142.250.159.100, ...
Connecting to docs.google.com (docs.google.com)|142.250.159.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-30-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/gqqfqvn22rrsk27dn8imkh5mrton4vet/1667197725000/15740322463873228959/*/1-4qx3IRyXUtb_QSItiWyIHZIo9hcBttg?e=download&uuid=3cd0caec-a8e7-472b-b451-745901f032de [following]
--2022-10-31 06:29:30--  https://doc-0o-30-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/gqqfqvn22rrsk27dn8imkh5mrton4vet/1667197725000/15740322463873228959/*/1-4qx3IRyXUtb_QSItiWyIHZIo9hcBttg?e=download&uuid=3cd0caec-a8e7-472b-b451-745901f032de
Resolving doc-0o-30-docs.googleusercontent.com (doc-0o-30-docs.googleusercontent.com)... 74.125.70.13

In [6]:
import pandas as pd

# 데이터 살펴보기
data = pd.read_csv(file_name, low_memory=False, index_col=0) 
first_date, last_date = data.loc[0,'date'], data.loc[data.shape[0]-1,'date']
print("수집한 게시물 범위 :", first_date, "~", last_date, "\n" \
      "수집된 데이터 크기 :", data.shape[0], "\n수집한 데이터 features :", data.columns)
data.tail()

수집한 게시물 범위 : 2021-03-03 ~ 2022-10-27 
수집된 데이터 크기 : 2470236 
수집한 데이터 features : Index(['title', 'author', 'date'], dtype='object')


,title,author,date
2470231,나 병신 ...,16125,2022-10-27
2470232,나고 닉 글 올리 규 싶 읃데,12779,2022-10-27
2470233,씨발 남자 직원 음식 깔아 주,5291,2022-10-27
2470234,나도 근데 아이디 두 번 바꿈,15378,2022-10-27
2470235,알바 바보 쉨 ㅋㅋㅋ 나도 차단 해봐라,16125,2022-10-27


In [7]:
## 혹시 모를, nan 값을 제거해주자. 제거해주지 않으면 float 이 발견되었다고 오류가 뜬다.
print(f"NAN 제거 전 데이터 수 = {data.shape}")
data = data[data.title.isna()==False]
data = data.reset_index(drop=True)
print(f"NAN 제거 후 데이터 수 = {data.shape}")

NAN 제거 전 데이터 수 = (2470236, 3)
NAN 제거 후 데이터 수 = (2469803, 3)


In [8]:
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## PMI 분석

출처 : [SoyNLP 공식문서](!https://github.com/lovit/soynlp/blob/master/tutorials/pmi_usage.ipynb)

이제 마지막 단계이다.

단어들 간의 연관관계를 분석해서 자살 관련 정보 파악에 도움이 되는 __다른 단어__들은 무엇이 있는 지 알아보자.

PMI 분석은

(word, contexts) 혹은 (input, outputs)의 상관성을 측정하는 방법으로, pmi 값이 클수록 두 변수의 상관성이 크다.

이를 위해서 특정 단어와 그 주위 단어들을 저장하는 행렬인 Word-Contexts Matrix를 생성한다.

아래 코드는 정제된 데이터를 통해서

word_contexts matrix를 가지고 있는 pmi 인스턴스를 생성한다.(인스턴스 생성은 3분정도 걸림)

생성된 pmi 인스턴스를 통해 수집한 데이터 속 단어들의 문맥과 유사 단어들을 알 수 있다.

In [9]:
from soynlp.word import pmi as pmi_func
from sklearn.metrics import pairwise_distances
from soynlp.vectorizer import sent_to_word_contexts_matrix
import numpy as np

def custom_tokenizer(sents) :
  # 그래프의 차원을 줄여주기 위해서 단어 길이가 1인 건 다 없애준다.
  return [word for word in sents.split() if len(word) > 1]

class Pmi() :

  def __init__(self, text_data) :

    self.mat, self.idx2vocab = sent_to_word_contexts_matrix(
        text_data,
        windows=3,
        min_tf=10,
        tokenizer= lambda x : custom_tokenizer(x), # (default) lambda x:x.split(),
        dynamic_weight=False,
        verbose=True
    )

    self.pmi, self.px, self.py = pmi_func(
        self.mat,
        min_pmi = 0,
        alpha = 0.0,
        beta = 0.75
    )

    self.vocab2idx = {vocab:idx for idx, vocab in enumerate(self.idx2vocab)}

  def context_words(self ,target, topk=10) :
    """
    :target: str
        String type query word
    :param topk: int
        Maximum number of similar items.
        If set top as negative value, it returns similarity with all words
    Returns
    ----------
    similars : list of tuple
        List contains tuples (item, cosine similarity)
        Its length is topk
    """
    query = self.vocab2idx[target]

    submatrix = self.pmi[query, :].tocsr() # get the row of query
    contexts = submatrix.nonzero()[1] # nonzero() return (rows, columns)
    pmi_i = submatrix.data

    most_relateds = [(idx, pmi_ij) for idx, pmi_ij in zip(contexts, pmi_i)]
    most_relateds = sorted(most_relateds, key=lambda x:-x[1])[:topk]
    most_relateds = [[idx, pmi_ij] for idx, pmi_ij in most_relateds]

    most_relateds = [(self.idx2vocab[idx], pmi_ij) for idx, pmi_ij in most_relateds]

    return most_relateds

  def similar_words(self, target, topk=10):
      """
      :target: str
          String type query word
      :param topk: int
          Maximum number of similar items.
          If set top as negative value, it returns similarity with all words
      Returns
      ----------
      similars : list of tuple
          List contains tuples (item, cosine similarity)
          Its length is topk
      """

      q = self.vocab2idx.get(target, -1)
      if q == -1:
          return []

      qvec = self.pmi[q].reshape(1,-1)
      dist = pairwise_distances(qvec, self.pmi, metric='cosine')[0]
      sim_idxs = dist.argsort()

      if topk > 0:
          sim_idxs = sim_idxs[:topk+1]

      results = [[idx, 1 - dist[idx]] for idx in sim_idxs if idx != q]
      # results = np.array(results)
      results = [(self.idx2vocab[idx], pmi_ij) for idx, pmi_ij in results]

      return results

  def np_context_words(self ,target, topk=10) :
    """
    효율적인 그래프 생성을 위한 코드.
    input(str) : a word
    ----------
    results : array of tuple
        List contains tuples (item, similarity)
        Its length is topk
    """
 
    submatrix = self.pmi[target, :].tocsr() # get the row of query
    contexts = submatrix.nonzero()[1] # nonzero() return (rows, columns)
    pmi_i = submatrix.data

    most_relateds = [(idx, pmi_ij) for idx, pmi_ij in zip(contexts, pmi_i)]
    most_relateds = sorted(most_relateds, key=lambda x:-x[1])[:topk]
    most_relateds = [[idx, pmi_ij] for idx, pmi_ij in most_relateds]
    results = np.array(most_relateds)
    # most_relateds = [(self.idx2vocab[idx], pmi_ij) for idx, pmi_ij in most_relateds]

    return results

  def np_similar_words(self, target, topk=10):
      """
      효율적인 그래프 생성을 위한 코드.
      input(str) : a word
      ----------
      results : array of tuple
          List contains tuples (item, cosine similarity)
          Its length is topk
      """

      qvec = self.pmi[target].reshape(1,-1)
      dist = pairwise_distances(qvec, self.pmi, metric='cosine')[0]
      sim_idxs = dist.argsort()

      if topk > 0:
          sim_idxs = sim_idxs[:topk+1]

      results = [[idx, 1 - dist[idx]] for idx in sim_idxs if idx != q]
      results = np.array(results)

      return results
  
  def idx_to_word(self, indices_list) :
    
    return [self.idx2vocab[index] for index in indices_list]

  def word_to_idx(self, word_list) :

    return [self.vocab2idx[word] for word in word_list]

pmi = Pmi(data.title)

Create (word, contexts) matrix
  - counting word frequency from 2469802 sents, mem=1.105 Gb
  - scanning (word, context) pairs from 2469802 sents, mem=1.593 Gb
  - (word, context) matrix was constructed. shape = (47605, 47605)                    
  - done


In [10]:
len(pmi.idx2vocab)

47605

# Graph 만들기

생성한 Pmi 인스턴스로, 

유사 단어 그래프와 문맥 단어 그래프를 만들어 보자.

### Graph 생성 알고리즘

```python
until recursion boundary :
  Seed -> Context_word_list
    for Context_word in Context_word_list
      build graph
```

그래프 생성에서 중요한 파라미터가 두가지 있다. 이 두가지를 크게 잡으면 좋겠지만, 그러다가 ram의 용량이 다차서 colab 세션이 다운될 수도 있으니 주의하자.
- max_recursion : 그래프 생성은 재귀의 형식이다, 재귀를 몇 번까지 허용할 것이냐를 정한다.
- graph_boundary : 웨이트를 몇까지 허용할 것이냐를 정한다. 



In [11]:
network = np.empty((len(pmi.idx2vocab),len(pmi.idx2vocab)), dtype=np.float16)
network.shape

(47605, 47605)

In [12]:
from scipy import sparse
from tqdm import tqdm
def make_network_array(node,current_recursion=0,graph_boundary=5,max_recursion=10,alpha=0.7) :

  global network

  context_words = pmi.np_context_words(node,topk=30) #context 뽑아오기
  context_words = context_words[np.where(context_words[:,1] > graph_boundary),:][0]

  if context_words.shape[0] == 0 :
    return []

  indices = np.array(context_words[:,0], dtype='int32')
  alpha = alpha**current_recursion

  network[int(node),indices]=context_words[:,1]*alpha
  # print(int(node), "-->", indices)
  if current_recursion <= max_recursion :
    for id in context_words[:,0] :
      make_network_array(id,
                        current_recursion=current_recursion+1,
                        graph_boundary=graph_boundary,
                        max_recursion=max_recursion)
  else :
    return []

seeds = ['자살','동반',
        '죽을래','투신','자해','죽고싶다','우울','살자',
        '인생','죽음']

graph_boundary = 3.5
max_recursion = 2

for seed in tqdm(pmi.word_to_idx(seeds)) :
    make_network_array(int(seed),
                  current_recursion=0,
                  graph_boundary=graph_boundary,
                  max_recursion=max_recursion)


from scipy.sparse import csr_matrix
network_name='network.npz'
sparse.save_npz(network_name,csr_matrix(network))

  0%|          | 0/10 [00:00<?, ?it/s]

3758


 10%|█         | 1/10 [00:12<01:56, 12.93s/it]

12003


 20%|██        | 2/10 [00:27<01:50, 13.77s/it]

...

 80%|████████  | 8/10 [02:09<00:29, 14.90s/it]

27


 90%|█████████ | 9/10 [02:22<00:14, 14.23s/it]

1095


100%|██████████| 10/10 [02:32<00:00, 15.26s/it]


In [ ]:
import networkx as nx

sparse_matrix = scipy.sparse.load_npz(network_name)
CG=nx.from_scipy_sparse_matrix(csr_network)

In [ ]:
# Weight 분포 보기
import pandas as pd 

def show_w_e_dist(G) :
  weights = [wt for (u, v, wt) in G.edges.data('weight')]
  degrees = [G.degree(node) for node in G.nodes]
  return pd.DataFrame(degrees), pd.DataFrame(weights)

ds, ws = show_w_e_dist(CG)
ds.describe()

In [ ]:
ws.describe()

In [ ]:
## idx -> word
relabel_nodes = {idx : word for idx, word in enumerate(pmi.idx2vocab)}
CG = nx.relabel_nodes(CG, relabel_nodes)

In [ ]:
## filter node which has less edges.
minimum_edges = 33
CG_fnodes = nx.subgraph_view(CG, 
                          filter_node=lambda x : CG.degree[x]>=minimum_edges) 
print(f"n(nodes) : {CG.number_of_nodes()} ---> {CG_fnodes.number_of_nodes()}")

In [ ]:
## filter edges which has less weights.

minimum_weight = 1.337664

def filter_edge(n1, n2):
    return CG_fnodes[n1][n2]['weight'] > minimum_weight

CG_fnodes_fedges = nx.subgraph_view(CG_fnodes, 
                                    filter_node = lambda x : True,
                                    filter_edge=filter_edge) 
print(f"n(edges) : {CG_fnodes.number_of_edges()} ---> {CG_fnodes_fedges.number_of_edges()}")


In [ ]:
page_rank_results = nx.pagerank(CG_fnodes_fedges)

In [ ]:
{key:rank for key,rank in sorted(page_rank_results.items(),key=lambda x:x[1],reverse=True) if len(key) > 1}

In [ ]:
seeds = ['자살', '자해', '동반', '우울', '죽음']
graph_boundary = 3
max_recursion = 15
ids = pmi.word_to_idx(seeds)
CG2 = nx.DiGraph()

for seed in tqdm(pmi.word_to_idx(seeds)) :
  CG2.add_node(seed)
  plot_graph(seed,
            current_recursion=0,
            graph_boundary=graph_boundary,
            max_recursion=max_recursion)

graph_name = './results/PMI_words_graph2.pickle'
# save graph object to file
pickle.dump(CG2, open(graph_name, 'wb'))

In [ ]:
CG = pickle.load(open(graph_name, 'rb'))

In [ ]:
# pos = nx.spring_layout(CG) 
# labels =nx.get_edge_attributes(CG,'weight')
# nx.draw_networkx_edge_labels(CG, pos, edge_labels = labels) 
nx.draw(CG, pos,with_labels = True, font_weight = "bold") 

In [ ]:
nx.write_gexf(CG_fnodes_fedges, "./models/context_graph_final.gexf") 

In [ ]:
from matplotlib import pylab
import networkx as nx

 def save_graph(graph,file_name):
    #initialze Figure
    plt.figure(num=None, figsize=(20, 20), dpi=80)
    plt.axis('off')
    fig = plt.figure(1)
    pos = nx.spring_layout(graph)
    nx.draw_networkx_nodes(graph,pos)
    nx.draw_networkx_edges(graph,pos)
    nx.draw_networkx_labels(graph,pos)

    cut = 1.00
    xmax = cut * max(xx for xx, yy in pos.values())
    ymax = cut * max(yy for xx, yy in pos.values())
    plt.xlim(0, xmax)
    plt.ylim(0, ymax)

    plt.savefig(file_name,bbox_inches="tight")
    pylab.close()
    del fig

#Assuming that the graph g has nodes and edges entered
save_graph(g,"my_graph.pdf")